In [ ]:
import pandas as pd

df = pd.read_table('list_landmarks_inshop.txt', sep = '\s+', skiprows = 1)

df

In [ ]:
s = 100

dfa = df[(df['clothes_type'] == 1) & (df['variation_type'] == 1) &
         (df['landmark_visibility_1'] == 0) & (df['landmark_visibility_2'] == 0) & 
         (df['landmark_visibility_3'] == 0) & (df['landmark_visibility_4'] == 0) &
         (df['landmark_visibility_5'] == 0) & (df['landmark_visibility_6'] == 0)][:s]

dfa

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(f)) for f in dfa['image_name']])

imgs.shape

In [ ]:
cols = [f'landmark_location_{t}_{i + 1}' for i in range(6) for t in ['x', 'y'] ]
cols

In [ ]:
dv = dfa[cols]

labels_t = dv.values.astype(int)
labels_t.shape

In [ ]:
def gen_labels(prob, around_func):
    res = np.zeros(imgs.shape[:-1] + (int(len(cols) / 2) + 1,))
    res[:, :, :, 0] = 1.0
    
    for i in range(len(res)):
        r = res[i]
        
        for j in range(0, len(labels_t[i]), 2):
            x = labels_t[i, j]
            y = labels_t[i, j + 1]
            c = int(j / 2) + 1
            
            for k in range(len(prob)):
                p = prob[k]
                
                for a in around_func(k):
                    ax = x + a[0]
                    ay = y + a[1]
                    
                    if ax >= 0 and ax < imgs.shape[2] and ay >= 0 and ay < imgs.shape[1]:
                        r[ay, ax, :] = 0.0
                        r[ay, ax, c] = p
                        r[ay, ax, 0] = 1.0 - p
    
    return res
    

In [ ]:
def around_square(n):
    return [(x, y) for x in range(-n, n + 1) for y in range(-n, n + 1) if abs(x) == n or abs(y) == n]

labels = gen_labels([1.0, 1.0, 1.0, 0.8, 0.8, 0.7, 0.7, 0.6, 0.6, 0.5], around_square)

labels.shape

In [ ]:
labels_t[0]

In [ ]:
labels[0, 59, 100:130]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

def imshow_label(index):
    plt.imshow(labels[index, :, :, 1:].sum(axis = -1), cmap = 'gray')


In [ ]:
imshow_label(0)

In [ ]:
imshow_label(1)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, UpSampling2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])

x = input

x = BatchNormalization()(x)

x = Conv2D(16, 3, padding='same', activation = 'relu')(x)
x = Conv2D(16, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(32, 3, padding='same', activation = 'relu')(x)
x = Conv2D(32, 3, padding='same', activation = 'relu')(x)
x = Conv2D(32, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(64, 3, padding='same', activation = 'relu')(x)
x = Conv2D(64, 3, padding='same', activation = 'relu')(x)
x = Conv2D(64, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(128, 3, padding='same', activation = 'relu')(x)
x = Conv2D(128, 3, padding='same', activation = 'relu')(x)
x = Conv2D(128, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(256, 3, padding='same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(128, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(128, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(64, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(32, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(16, 3, padding = 'same', activation = 'relu')(x)
x = Conv2DTranspose(16, 3, padding = 'same', activation = 'relu')(x)

x = Dropout(0.3)(x)

output = Dense(labels.shape[-1], activation = 'softmax')(x)

model = Model(inputs = input, outputs = output)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

model.summary()

In [ ]:
from keras.models import load_model

model = load_model('model/c2_0.h5')

In [ ]:
wg = np.ones(labels.shape[-1]) * (imgs.shape[1] * imgs.shape[2])
wg[0] = 1

print(wg)

hist = model.fit(imgs, labels, initial_epoch = 0, epochs = 100, batch_size = 10, class_weight = wg, validation_split = 0.2)

hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (16., 4.)

plt.subplot(1, 4, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 4, 2)
plt.plot(hist.history['acc'])

plt.subplot(1, 4, 3)
plt.plot(hist.history['val_loss'])

plt.subplot(1, 4, 4)
plt.plot(hist.history['val_acc'])

In [ ]:
model.save('model/c2_100.h5')

In [ ]:
import cv2

colors = [(255, 255, 255), (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (255, 165, 0), (210, 180, 140)]

def predict(index, s = 5.0, ns = 5, c_size = 5):
    plt.rcParams['figure.figsize'] = (s * 4, s)
    
    img = imgs[index]

    p = model.predict(np.array([img]))[0]

    img1 = np.apply_along_axis(lambda x: colors[x.argmax()], -1, labels[index])
    img2 = np.apply_along_axis(lambda x: colors[x.argmax()], -1, p)
    
    img3 = cv2.addWeighted(img.astype(int), 0.4, img2, 0.6, 0)
    
    plt.subplot(1, 4, 1)
    plt.imshow(img1)
    
    plt.subplot(1, 4, 2)
    plt.imshow(img3)
    
    plt.subplot(1, 4, 3)
    plt.imshow(img2)

    img4 = img.astype(int)
    
    for i in range(1, labels.shape[-1]):
        print(f"--- {i} : {labels_t[index, (i - 1) * 2]}, {labels_t[index, (i - 1) * 2 + 1]} ---")

        t = p[:, :, i]
        fst = True
        
        for j in np.argsort(t.flatten())[::-1][:ns]:
            y, x = divmod(j, imgs.shape[2])
        
            print(f"{j}, x={x}, y={y}, prob={t[y, x]}")
            
            if fst:
                img4 = cv2.circle(img4, (x, y), c_size, colors[i], -1)
                fst = False

    plt.subplot(1, 4, 4)
    plt.imshow(img4)

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict(10)

In [ ]:
predict(15)

In [ ]:
predict(-1)

In [ ]:
predict(-2)

In [ ]:
predict(-3)